In [1]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [2]:
import ragna

ragna.__version__

platform/c++/implementation/internal.cpp:205:reinit_singlethreaded(): Reinitialising as single-threaded.


'0.1.dev29+g03721c7.d20231009084220'

In [4]:
from ragna import Config

config = Config()
config

Config(local_cache_root=PosixPath('~/.cache/ragna'), rag=RagConfig(database_url='memory', queue_url='memory', document=<class 'ragna.core.LocalDocument'>, source_storages=[<class 'ragna.source_storages.RagnaDemoSourceStorage'>], assistants=[<class 'ragna.assistants.RagnaDemoAssistant'>]), api=ApiConfig(url=Url('http://127.0.0.1:31476/'), upload_token_secret=SecretStr('**********'), upload_token_ttl=300), ui=UiConfig(url=Url('http://127.0.0.1:31476/')))

In [5]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [10]:
from ragna.assistants import Gpt4, Gpt35Turbo16k, RagnaDemoAssistant
from ragna.core import Rag
from ragna.source_storages import Chroma, LanceDB, RagnaDemoSourceStorage

rag = Rag(config)

async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

I can't really help you with your prompt:

> What is Ragna?

I can at least show you the sources that I was given:

- demo_document.txt: Ragna is an open-source RAG orchestration app


In [11]:
import asyncio
import itertools
from pprint import pprint

source_storages = [Chroma, LanceDB]
assistants = [Gpt35Turbo16k, Gpt4]


async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f66e654ccc0>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f66e654c640>,
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f66e654c540>,
 ('LanceDB', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f66e65cd640>}


In [12]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): 'Ragna is an open-source RAG '
                                         '(Retrieve, Analyze, Generate) '
                                         'orchestration app. It is designed to '
                                         'assist in the creation and '
                                         'management of conversational AI '
                                         'models. Ragna provides a '
                                         'user-friendly interface for '
                                         'developers to retrieve data, analyze '
                                         'it, and generate responses. It is '
  